# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,283.21,62,75,12.35,CL,1728769553
1,1,talnakh,69.4865,88.3972,267.17,94,100,4.20,RU,1728769316
2,2,iqaluit,63.7506,-68.5145,277.00,80,75,4.63,CA,1728769555
3,3,papatowai,-46.5619,169.4708,279.72,82,99,3.20,NZ,1728769556
4,4,edinburgh of the seven seas,-37.0676,-12.3116,286.23,76,57,4.68,SH,1728769558


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": city_data_df['Lat'],
    "longitude": city_data_df['Lng'],
    "City": city_data_df['City'],
    "Humidity": city_data_df['Humidity']
}
city_df = pd.DataFrame(coordinates)


map_plot_1 = city_df.hvplot.points(
    "longitude",
    "latitude",
    size = 'Humidity',
    color = 'City',
    geo = True,
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df['Max Temp'] > 280) & (city_data_df['Humidity']<90) & (city_data_df['Humidity']>50)

# Drop any rows with null values
city_preferred = city_data_df[criteria].dropna()

# Display sample data
city_preferred.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,283.21,62,75,12.35,CL,1728769553
4,4,edinburgh of the seven seas,-37.0676,-12.3116,286.23,76,57,4.68,SH,1728769558
6,6,broome,42.2506,-75.8330,287.40,59,72,1.95,US,1728769560
11,11,mazagao,-0.1150,-51.2894,302.25,61,65,4.86,BR,1728769566
14,14,adamstown,-25.0660,-130.1015,294.33,81,98,4.85,PN,1728769570
18,18,windsor,42.3001,-83.0165,291.20,63,0,4.12,CA,1728769325
24,24,smithers,54.7804,-127.1743,286.33,54,75,4.12,CA,1728769581
26,26,urangan,-25.3000,152.9000,294.55,66,97,6.48,AU,1728769583
27,27,caleta de carquin,-11.0925,-77.6267,292.22,79,76,4.22,PE,1728769584
29,29,port mathurin,-19.6833,63.4167,295.54,71,100,9.74,MU,1728769587


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_preferred[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(15)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,punta arenas,CL,-53.1500,-70.9167,62,
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,76,
6,broome,US,42.2506,-75.8330,59,
11,mazagao,BR,-0.1150,-51.2894,61,
14,adamstown,PN,-25.0660,-130.1015,81,
18,windsor,CA,42.3001,-83.0165,63,
24,smithers,CA,54.7804,-127.1743,54,
26,urangan,AU,-25.3000,152.9000,66,
27,caleta de carquin,PE,-11.0925,-77.6267,79,
29,port mathurin,MU,-19.6833,63.4167,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": 'accommodation',
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
punta arenas - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: Sea View
broome - nearest hotel: No hotel found
mazagao - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
windsor - nearest hotel: Marine City Apartments
smithers - nearest hotel: Sunshine Inn Hotel
urangan - nearest hotel: YHA Hervey Bay
caleta de carquin - nearest hotel: Atlantic
port mathurin - nearest hotel: Résidence Véronia
hermanus - nearest hotel: Aloe House
blackmans bay - nearest hotel: Villa Howden
namibe - nearest hotel: Pensão Delta do Deserto
olinala - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
ipameri - nearest hotel: No hotel found
at taraf - nearest hotel: منازل عائلة المطلق بالعمران الاحساء
jamestown - nearest hotel: DoubleTree Jamestown
kingston - nearest hotel: Aphrodite Hotel
puerto natales - nearest hotel: Hostal Dorotea
laguna - nearest hotel: Holiday Inn Express & Suites
dunda - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,punta arenas,CL,-53.1500,-70.9167,62,No hotel found
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,76,Sea View
6,broome,US,42.2506,-75.8330,59,No hotel found
11,mazagao,BR,-0.1150,-51.2894,61,No hotel found
14,adamstown,PN,-25.0660,-130.1015,81,No hotel found
...,...,...,...,...,...,...
566,howard springs,AU,-12.4970,131.0470,83,Darwin Boomerang Motel and Caravan Park
574,big bend,SZ,-26.8167,31.9333,89,No hotel found
575,qarah,SY,34.1527,36.7415,59,No hotel found
577,mlangali,TZ,-9.7833,34.5167,77,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": hotel_df['Lat'],
    "longitude": hotel_df['Lng'],
    "City": hotel_df['City'],
    "Country": hotel_df['Country'],
    "Hotel": hotel_df['Hotel Name']
}
hotel_map_df = pd.DataFrame(coordinates)

hover_cols = ['City','Country','Hotel']

map_plot_2 = hotel_map_df.hvplot.points(
    "longitude",
    "latitude",
    color = 'City',
    geo = True,
    hover_cols=hover_cols,
    tiles = "OSM"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Country,Hotel)